In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.functions import col, from_json, regexp_replace, to_timestamp

# -----------------------------------------------
# STEP 1: Definir esquemas
# -----------------------------------------------

# Esquema para los datos planos dentro de 'current'
pago_schema = StructType([
    StructField("Id", IntegerType()),
    StructField("ClienteId", StringType()),
    StructField("Monto", StringType()),  # money como string para evitar problemas
    StructField("MetodoPago", StringType()),
    StructField("FechaPago", StringType()),  # datetime como string para parsear luego
    StructField("Estado", StringType())
])

# Esquema para el CloudEvent recibido
cloud_event_schema = StructType([
    StructField("specversion", StringType()),
    StructField("type", StringType()),
    StructField("source", StringType()),
    StructField("id", StringType()),
    StructField("time", StringType()),
    StructField("datacontenttype", StringType()),
    StructField("operation", StringType()),
    StructField("data", StringType())  # JSON anidado como string
])

# Esquema para el campo 'data' dentro del CloudEvent
data_schema = StructType([
    StructField("eventsource", StructType([
        StructField("db", StringType()),
        StructField("schema", StringType()),
        StructField("tbl", StringType()),
        StructField("cols", ArrayType(StructType([
            StructField("name", StringType()),
            StructField("type", StringType()),
            StructField("index", StringType())
        ]))),
        StructField("pkkey", ArrayType(StructType([
            StructField("columnname", StringType()),
            StructField("value", StringType())
        ]))),
        StructField("transaction", StructType([
            StructField("commitlsn", StringType()),
            StructField("beginlsn", StringType()),
            StructField("sequencenumber", StringType()),
            StructField("committime", StringType())
        ]))
    ])),
    StructField("eventrow", StructType([
        StructField("old", StringType()),
        StructField("current", StringType())
    ]))
])

# -----------------------------------------------
# STEP 2: Configuración para leer desde Azure Event Hub
# -----------------------------------------------

event_hub_conn_str = "Endpoint=sb://arquitecturadatosdemoces.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=Oa8J/Zenf+jtntTTW0CZbgj3dow0WLaz8+AEhDgfUf0=;EntityPath=pagos_ces"

eh_conf = {
    'eventhubs.connectionString': sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(event_hub_conn_str)
}

raw_df = (spark.readStream
    .format("eventhubs")
    .options(**eh_conf)
    .load()
)

# -----------------------------------------------
# STEP 3: Decodificación y parseo de los mensajes
# -----------------------------------------------

# Parsear CloudEvent
parsed_df = raw_df.withColumn("body_str", col("body").cast("string"))
parsed_df = parsed_df.withColumn("cloud_event", from_json("body_str", cloud_event_schema))
# Parsear campo 'data' JSON anidado
parsed_df = parsed_df.withColumn("data_json", from_json(col("cloud_event.data"), data_schema))

# Desescapar el string JSON en 'eventrow.old' para poder parsearlo correctamente
clean_current = regexp_replace(col("data_json.eventrow.current"), r'\\', '')
# Parsear el JSON limpio con el esquema final
final_df = final_df.withColumn("current_data", from_json(clean_current, pago_schema))
# 3. Seleccionar los campos deseados y convertir tipos
result_df = final_df.select(
    col("current_data.Id").cast(IntegerType()).alias("Id"),
    col("current_data.ClienteId").alias("ClienteId"),
    col("current_data.Monto").cast("double").alias("Monto"),
    col("current_data.MetodoPago"),
    to_timestamp(col("current_data.FechaPago"), "yyyy-MM-dd HH:mm:ss.SSSSSSS").alias("FechaPago"),
    col("current_data.Estado"),
    col("cloud_event.time").alias("eventhub_time"),
    col("enqueuedTime"),
    lit("eventhub").alias("source")
)

# 4. Calcular diferencia de tiempo
result_df = result_df.withColumn(
    "tiempo_diferencia_segundos",
    (col("enqueuedTime").cast("long") - col("FechaPago").cast("long"))
)

# Mostrar resultado
result_df.display()


